# Project

## Load libraries and setup environment

In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# authenticate
auth.authenticate_user()

# Set up environment variables
project_id = 'fast-oxide-395508'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'

## Get dataset

### Arterial hypoxemia (arterial oxygen tension [Pa02]/fraction of inspired oxygen [FiO2] <300)  
Module hosp doesn't have the these data, so we get data from module icu

In [41]:
# Module icu:
query = """
SELECT *
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%inspired%" OR LOWER(label) LIKE "%arterial%pressure" OR LOWER(label) LIKE "%base%excess%" OR LOWER(label) LIKE "%urine%"
"""

df = run_query(query)
print(df)

    itemid                         label                  abbreviation  \
0   225454                 Urine Culture                 Urine Culture   
1   220224          Arterial O2 pressure                PO2 (Arterial)   
2   220235         Arterial CO2 Pressure               PCO2 (Arterial)   
3   220799     ZSpecific Gravity (urine)     ZSpecific gravity (urine)   
4   224828          Arterial Base Excess          Arterial Base Excess   
5   227471      Specific Gravity (urine)      Specific Gravity (urine)   
6   224015                  Urine Source                  Urine Source   
7   224016                   Urine Color                   Urine Color   
8   224876              Urine Appearance              Urine Appearance   
9   226566     Urine and GU Irrigant Out     Urine and GU Irrigant Out   
10  226627                      OR Urine                      OR Urine   
11  226631                    PACU Urine                    PACU Urine   
12  227489  GU Irrigant/Urine Volume O

We find "Arterial O2 pressure" and "Inspired O2 Fraction" is what we want

In [ ]:
query = f"""
SELECT
  d1.subject_id,
  d1.hadm_id,
  d1.stay_id,
  d1.itemid,
  d1.valuenum AS PaO2,
  d1.valueuom,
  d2.valuenum AS FiO2,
  (d1.valuenum / NULLIF(d2.valuenum, 0)) AS ratio
FROM
  (
    SELECT *
    FROM `physionet-data.mimiciv_icu.chartevents`
    WHERE itemid = 220224
  ) d1
JOIN
  (
    SELECT *
    FROM `physionet-data.mimiciv_icu.chartevents`
    WHERE itemid = 223835
  ) d2
ON
  d1.subject_id = d2.subject_id
AND
  d1.hadm_id = d2.hadm_id
AND
  d1.stay_id = d2.stay_id
WHERE
  (d1.valuenum / NULLIF(d2.valuenum, 0)) < 300
"""

df = run_query(query)
Arterial_hypoxemia_subject_id = df['subject_id'].unique()
Arterial_hypoxemia_subject_id = pd.DataFrame(Arterial_hypoxemia_subject_id, columns=['subject_id'])
Arterial_hypoxemia_subject_id

,subject_id
0,12627553
1,15199651
2,13632970
3,15620117
4,12889874
...,...
24309,19827412
24310,10267064
24311,12538508
24312,16605907


In [46]:
subject_ids = patients_cohort['subject_id'].tolist()

# 先从outputevents中获取相关数据
query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.outputevents`
WHERE itemid = 226627 AND subject_id IN ({','.join(map(str, subject_ids))})
"""

df = run_query(query)

# 在Python中进行合并和筛选
df = df.merge(patients_cohort[['subject_id', 'weaning_start_time']], on="subject_id", how="inner")

# 筛选出weaning_start_time前24小时的数据
mask = (df['charttime'] > df['weaning_start_time'] - pd.Timedelta(hours=24)) & (df['charttime'] <= df['weaning_start_time'])
df_filtered = df[mask]
df_filtered

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valueuom,weaning_start_time
0,10004422,21255400,32155744,87605,2111-01-17 19:11:00,2111-01-17 19:11:00,226627,300.0,ml,2111-01-18 06:31:00
1,10004457,23251352,31494479,53913,2141-12-17 15:00:00,2141-12-17 17:09:00,226627,675.0,ml,2141-12-17 18:04:00
2,10009628,25926192,35258379,86518,2153-09-19 13:40:00,2153-09-19 14:08:00,226627,720.0,ml,2153-09-19 18:18:00
5,10014729,28889419,33558396,53913,2125-02-27 14:45:00,2125-02-27 17:38:00,226627,415.0,ml,2125-02-27 19:05:00
6,10017437,28440971,33439506,12098,2118-04-17 15:03:00,2118-04-17 15:03:00,226627,550.0,ml,2118-04-17 21:00:00
...,...,...,...,...,...,...,...,...,...,...
3476,19968039,20687784,35528180,91879,2132-04-20 23:21:00,2132-04-20 23:21:00,226627,1000.0,ml,2132-04-21 10:45:00
3477,19972172,23625602,39366389,49403,2156-10-20 22:40:00,2156-10-21 03:06:00,226627,100.0,ml,2156-10-21 14:39:00
3478,19975044,20044208,37790066,28029,2155-02-24 15:51:00,2155-02-24 15:51:00,226627,1250.0,ml,2155-02-24 21:47:00
3480,19984052,28687121,33735777,28791,2184-11-12 13:00:00,2184-11-12 15:50:00,226627,670.0,ml,2184-11-12 17:07:00


### Arterial hypotension (eg, systolic blood pressure (SBP] <90 mmHg, mean arterial pressure (MAP) <70mmHg)

In [ ]:
query = f"""
SELECT label, itemid
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%systolic%"
"""

df = run_query(query)
df

,label,itemid
0,Aortic Pressure Signal - Systolic,228152
1,Pulmonary Atrtery Pressure Signal - Systolic,229669
2,Left Ventricular Pressure Signal - Systolic,229899
3,Pulmonary Artery Pressure systolic,220059
4,RV systolic pressure(PA Line),226850
5,PA systolic pressure(PA Line),226852
6,Arterial Blood Pressure systolic,220050
7,Non Invasive Blood Pressure systolic,220179
8,Manual Blood Pressure Systolic Left,224167
9,ART BP Systolic,225309


We choose "Non Invasive Blood Pressure systolic"

In [ ]:
query = f"""
SELECT label, itemid
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%arterial%mean%" OR LOWER(label) LIKE "%oxygenation%index%"
"""

df = run_query(query)
df

,label,itemid
0,Arterial Blood Pressure mean,220052


We choose "Arterial Blood Pressure mean"

In [ ]:
query = f"""
SELECT
  t1.subject_id,
  t1.hadm_id,
  t1.stay_id,
  t1.itemid,
  t1.valuenum AS SBP,
  t1.valueuom,
  t2.itemid,
  t2.valuenum AS MAP,
  t2.valueuom
FROM
  (
    SELECT *
    FROM `physionet-data.mimiciv_icu.chartevents`
    WHERE itemid = 220179 AND valuenum < 90
  ) t1
JOIN
  (
    SELECT *
    FROM `physionet-data.mimiciv_icu.chartevents`
    WHERE itemid = 220052 AND valuenum < 70
  ) t2
ON
  t1.subject_id = t2.subject_id
AND
  t1.hadm_id = t2.hadm_id
AND
  t1.stay_id = t2.stay_id
"""

df = run_query(query)
Arterial_hypotension_subject_id = df['subject_id'].unique()
Arterial_hypotension_subject_id = pd.DataFrame(Arterial_hypotension_subject_id, columns=['subject_id'])
Arterial_hypotension_subject_id

,subject_id
0,13612783
1,18756985
2,15112972
3,17030962
4,19097438
...,...
11258,13783676
11259,13730292
11260,19889178
11261,18994071


### Leukocytosis (white blood cell (WBC] count >12,000 microL-1) or leukopenia (WBC count <4000microL-1)

In [60]:
query = f"""
SELECT label, itemid
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%wbc%" OR LOWER(label) LIKE "%anion%"
"""

df = run_query(query)
df

,label,itemid
0,WBC,220546
1,Anion gap,227073
2,WbcApacheIIScore,226779
3,WBCApacheIIValue,226780
4,WBC_ApacheIV,227062
5,WBCScore_ApacheIV,227063


We choose "WBC"

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 220546 AND (valuenum > 12000 OR valuenum < 4000)
"""

df = run_query(query)
Leukocytosis_subject_id = df['subject_id'].unique()
Leukocytosis_subject_id = pd.DataFrame(Leukocytosis_subject_id, columns=['subject_id'])
Leukocytosis_subject_id

,subject_id
0,16433404
1,17589125
2,18680755
3,19650110
4,10437898
...,...
49475,11494577
49476,10783607
49477,19946367
49478,17888905


### Thrombocytopenia (platelet count <100,000 microL-1)

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%platelet%" or LOWER(label) LIKE "%creatinine%"
"""

df = run_query(query)
df

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220615,Creatinine (serum),Creatinine (serum),chartevents,Labs,None,Numeric,NaN,NaN
1,225678,ZPlatelet Count,ZPlatelet Count,chartevents,Labs,None,Numeric,NaN,NaN
2,229761,Creatinine (whole blood),Creatinine (whole blood),chartevents,Labs,None,Numeric,NaN,NaN
3,227457,Platelet Count,Platelet Count,chartevents,Labs,None,Numeric with tag,NaN,NaN
4,226751,CreatinineApacheIIScore,CreatinineApacheIIScore,chartevents,Scores - APACHE II,None,Numeric,NaN,NaN
5,226752,CreatinineApacheIIValue,CreatinineApacheIIValue,chartevents,Scores - APACHE II,None,Numeric,NaN,NaN
6,227005,Creatinine_ApacheIV,Creatinine_ApacheIV,chartevents,Scores - APACHE IV (2),None,Numeric,NaN,NaN
7,225170,Platelets,Platelets,inputevents,Blood Products/Colloids,mL,Solution,NaN,NaN
8,226369,OR Platelet Intake,OR Platelet Intake,inputevents,Blood Products/Colloids,mL,Solution,NaN,NaN
9,227071,PACU Platelet Intake,PACU Platelet Intake,inputevents,Blood Products/Colloids,mL,Solution,NaN,NaN


We choose "Platelet Count"

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 227457 AND valuenum < 100000
"""

df = run_query(query)
Thrombocytopenia_subject_id = df['subject_id'].unique()
Thrombocytopenia_subject_id = pd.DataFrame(Thrombocytopenia_subject_id, columns=['subject_id'])
Thrombocytopenia_subject_id

,subject_id
0,17914766
1,18259787
2,16178416
3,17924051
4,18151060
...,...
49485,19157537
49486,17795151
49487,12493025
49488,10641032


### Heart rate >90 beats/min

In [ ]:
query = f"""
SELECT label, itemid
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%heart%rate%" OR LOWER(label) LIKE "%respiratory%rate%"
"""

df = run_query(query)
df

,label,itemid
0,Heart rate Alarm - High,220046
1,Heart Rate Alarm - Low,220047
2,Respiratory Rate,220210
3,Respiratory Rate (Set),224688
4,Respiratory Rate (spontaneous),224689
5,Respiratory Rate (Total),224690
6,Heart Rate,220045


We choose "Heart Rate"

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 220045 AND valuenum > 90
"""

df = run_query(query)
Heart_rate_subject_id = df['subject_id'].unique()
Heart_rate_subject_id = pd.DataFrame(Heart_rate_subject_id, columns=['subject_id'])
Heart_rate_subject_id

,subject_id
0,10390732
1,16748272
2,10440899
3,17167982
4,18595258
...,...
41622,15591535
41623,10911585
41624,16774594
41625,15181772


### patients who have invasive Ventilation

In [ ]:
query = f"""
SELECT char.*, dlab.label
FROM `physionet-data.mimiciv_icu.procedureevents` AS char
INNER JOIN `physionet-data.mimiciv_icu.d_items` AS dlab
 ON char.itemid = dlab.itemid
WHERE char.itemid = 225792
ORDER BY subject_id
"""

df_invasive_ventilation = run_query(query)
invasive_ventilation_subject_id = df_invasive_ventilation['subject_id'].unique()
invasive_ventilation_subject_id = pd.DataFrame(invasive_ventilation_subject_id, columns=['subject_id'])
df_invasive_ventilation

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,ORIGINALAMOUNT,ORIGINALRATE,label
0,10001884,26184834,37510196,42150,2131-01-11 04:40:00,2131-01-12 17:40:00,2131-01-12 17:49:00.000,225792,2220.0,min,...,3830120,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,2220.0,1.0,Invasive Ventilation
1,10001884,26184834,37510196,91332,2131-01-13 04:00:00,2131-01-19 17:45:00,2131-01-19 18:44:00.000,225792,9465.0,min,...,4465887,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,9465.0,1.0,Invasive Ventilation
2,10001884,26184834,37510196,91332,2131-01-15 04:07:00,2131-01-19 17:43:00,2131-01-19 18:44:00.000,225792,6576.0,min,...,1861924,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,6576.0,1.0,Invasive Ventilation
3,10002013,23581541,39060235,27479,2160-05-18 14:19:00,2160-05-18 18:01:00,2160-05-18 18:39:00.000,225792,222.0,min,...,4169380,Ventilation,ContinuousProcess,96.0,1,0,FinishedRunning,222.0,1.0,Invasive Ventilation
4,10002428,23473524,35479615,27479,2156-05-11 16:05:00,2156-05-20 10:45:00,2156-05-20 10:51:00.000,225792,12640.0,min,...,3976442,Ventilation,ContinuousProcess,48.4,1,0,FinishedRunning,12640.0,1.0,Invasive Ventilation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30705,19999068,21606769,30143796,29141,2161-08-25 15:35:00,2161-08-28 13:36:00,2161-08-28 13:36:00.000,225792,4201.0,min,...,858585,Ventilation,ContinuousProcess,55.8,1,0,FinishedRunning,4201.0,1.0,Invasive Ventilation
30706,19999287,20175828,35165301,15882,2197-08-06 19:29:00,2197-08-07 09:20:00,2197-08-07 09:23:00.000,225792,831.0,min,...,9514927,Ventilation,ContinuousProcess,86.2,1,0,FinishedRunning,831.0,1.0,Invasive Ventilation
30707,19999442,26785317,32336619,86640,2148-11-19 19:00:00,2148-11-20 14:15:00,2148-11-20 14:17:00.000,225792,1155.0,min,...,9582145,Ventilation,ContinuousProcess,107.5,1,0,FinishedRunning,1155.0,1.0,Invasive Ventilation
30708,19999840,21033226,38978960,<NA>,2164-09-12 09:41:00,2164-09-17 16:35:00,2164-09-17 16:35:22.453,225792,7614.0,min,...,7234916,Ventilation,ContinuousProcess,77.5,1,0,FinishedRunning,7614.0,1.0,Invasive Ventilation


### merge all the above filters to get the final cohort of Sepsis patients

In [ ]:
merged_df = Arterial_hypotension_subject_id.merge(invasive_ventilation_subject_id, on='subject_id', how='inner')
merged_df = merged_df.merge(Leukocytosis_subject_id, on='subject_id', how='inner')
merged_df = merged_df.merge(Thrombocytopenia_subject_id, on='subject_id', how='inner')
#merged_df = merged_df.merge(Heart_rate_subject_id, on='subject_id', how='inner')
sepsis_subject_ids = merged_df['subject_id'].unique()

print(sepsis_subject_ids)

<IntegerArray>
[13612783, 18756985, 15112972, 17030962, 19097438, 10003400, 19461458,
 18346662, 19420664, 10813665,
 ...
 17355264, 11152474, 11045637, 12882485, 17809884, 13783676, 13730292,
 19889178, 18994071, 16160074]
Length: 11247, dtype: Int64


exclude patients under 18 and repeated ICU admission

In [ ]:
query = """
SELECT subject_id
FROM `physionet-data.mimiciv_hosp.patients`
WHERE anchor_age >= 18
"""

age_filtered_df = run_query(query)

final_df = merged_df.merge(age_filtered_df, on='subject_id', how='inner')
final_df

,subject_id
0,10098993
1,17559288
2,16467047
3,11750559
4,18454868
...,...
9434,11230804
9435,18482037
9436,15276613
9437,17411386


In [ ]:
query = """
SELECT subject_id
FROM `physionet-data.mimiciv_icu.icustays`
GROUP BY subject_id
HAVING COUNT(*) > 1
"""

multiple_admission_subjects = run_query(query)

final_df_2 = final_df[~final_df['subject_id'].isin(multiple_admission_subjects['subject_id'])]
final_df_2

,subject_id
2,16467047
5,17087880
6,19556965
9,10222191
10,10252795
...,...
9430,11102902
9431,14477446
9434,11230804
9436,15276613


In [ ]:
final_df_2.to_csv('filtered_patients_final.csv', index=False)

### weaning success

#### No intubation or invasive ventilation within 48 h after weaning, and noninvasive ventilation time was shorter than 48 h after weaning

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%wean%"
"""

df = run_query(query)
df

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,223870,Wean,Wean,chartevents,Respiratory,None,Text,NaN,NaN


In [ ]:
query = f"""
SELECT char.*
FROM `physionet-data.mimiciv_icu.chartevents` AS char
WHERE char.itemid = 223870
ORDER BY subject_id
"""

df_weaning = run_query(query)
df_weaning

,subject_id,hadm_id,stay_id,caregiver_id,charttime,storetime,itemid,value,valuenum,valueuom,warning


用"Wean"找不到数据

In [ ]:
query = f"""
SELECT label, itemid, linksto
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%extubation%"
"""

df = run_query(query)
df

,label,itemid,linksto
0,Unplanned Extubation (patient-initiated),225468,procedureevents
1,Unplanned Extubation (non-patient initiated),225477,procedureevents
2,Extubation,227194,procedureevents


In [ ]:
query = f"""
SELECT label, itemid, linksto
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%invasive ventilation%" OR LOWER(label) LIKE "%intubation%"
"""

df = run_query(query)
df

,label,itemid,linksto
0,Intubation,224385,procedureevents
1,Invasive Ventilation,225792,procedureevents
2,Non-invasive Ventilation,225794,procedureevents
3,Mallampati classification (Intubation),222871,chartevents
4,Induction/Drugs/Doses (Intubation),225271,chartevents
5,Indication (Intubation),225291,chartevents
6,Mouth Opening (Intubation),225294,chartevents
7,Thyromental Distance (Intubation),225295,chartevents
8,Mandibular Prognatism (Intubation),225296,chartevents
9,Teeth/Dentition (Intubation),225297,chartevents


We choose "Extubation", "Intubation" and "Invasive Ventilation"

Extubation

In [ ]:
query = f"""
SELECT char.*, dlab.label
FROM `physionet-data.mimiciv_icu.procedureevents` AS char
INNER JOIN `physionet-data.mimiciv_icu.d_items` AS dlab
 ON char.itemid = dlab.itemid
WHERE char.itemid = 227194
ORDER BY subject_id
"""

df_weaning = run_query(query)
df_weaning

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,ORIGINALAMOUNT,ORIGINALRATE,label
0,10001884,26184834,37510196,35966,2131-01-12 17:40:00,2131-01-12 17:41:00,2131-01-12 17:50:00,227194,1.0,None,...,6470885,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0.0,Extubation
1,10002013,23581541,39060235,65197,2160-05-18 18:00:00,2160-05-18 18:01:00,2160-05-18 17:59:00,227194,1.0,None,...,3546453,Intubation/Extubation,Task,96.0,0,0,FinishedRunning,1.0,0.0,Extubation
2,10002428,23473524,35479615,42665,2156-05-20 10:45:00,2156-05-20 10:46:00,2156-05-20 10:51:00,227194,1.0,None,...,9794232,Intubation/Extubation,Task,48.4,0,0,FinishedRunning,1.0,0.0,Extubation
3,10002428,28662225,38875437,3009,2156-04-22 17:10:00,2156-04-22 17:11:00,2156-04-22 17:10:00,227194,1.0,None,...,9954431,Intubation/Extubation,Task,43.0,0,0,FinishedRunning,1.0,0.0,Extubation
4,10002760,28094813,31831386,50512,2141-04-21 08:00:00,2141-04-21 08:01:00,2141-04-21 08:04:00,227194,1.0,None,...,2857656,Intubation/Extubation,Task,91.0,0,0,FinishedRunning,1.0,0.0,Extubation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22331,19998843,24842066,30988867,93405,2187-02-08 17:09:00,2187-02-08 17:10:00,2187-02-08 17:15:00,227194,1.0,None,...,9357731,Intubation/Extubation,Task,90.0,0,0,FinishedRunning,1.0,0.0,Extubation
22332,19999068,21606769,30143796,99850,2161-08-28 13:35:00,2161-08-28 13:36:00,2161-08-28 13:37:00,227194,1.0,None,...,489079,Intubation/Extubation,Task,55.8,0,0,FinishedRunning,1.0,0.0,Extubation
22333,19999287,20175828,35165301,50512,2197-08-07 09:20:00,2197-08-07 09:21:00,2197-08-07 09:23:00,227194,1.0,None,...,4142066,Intubation/Extubation,Task,86.2,0,0,FinishedRunning,1.0,0.0,Extubation
22334,19999442,26785317,32336619,26402,2148-11-20 14:15:00,2148-11-20 14:16:00,2148-11-20 14:17:00,227194,1.0,None,...,8029280,Intubation/Extubation,Task,107.5,0,0,FinishedRunning,1.0,0.0,Extubation


Intubation

In [ ]:
query = f"""
SELECT char.*, dlab.label
FROM `physionet-data.mimiciv_icu.procedureevents` AS char
INNER JOIN `physionet-data.mimiciv_icu.d_items` AS dlab
 ON char.itemid = dlab.itemid
WHERE char.itemid = 224385
ORDER BY subject_id
"""

df_intubation = run_query(query)
df_intubation

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,ORIGINALAMOUNT,ORIGINALRATE,label
0,10001884,26184834,37510196,42150,2131-01-11 04:30:00,2131-01-11 04:31:00,2131-01-11 05:40:00,224385,1.0,None,...,2338046,Intubation/Extubation,Task,65.0,0,0,FinishedRunning,1.0,0.0,Intubation
1,10003400,23559586,38383343,36212,2137-08-17 21:21:00,2137-08-17 21:22:00,2137-08-17 21:21:00,224385,1.0,None,...,9156137,Intubation/Extubation,Task,90.5,0,0,FinishedRunning,1.0,0.0,Intubation
2,10004401,27939719,31202136,58760,2144-04-11 05:03:00,2144-04-11 05:04:00,2144-04-11 05:03:00,224385,1.0,None,...,991564,Intubation/Extubation,Task,120.0,0,0,FinishedRunning,1.0,0.0,Intubation
3,10004401,29988601,32773003,38908,2144-01-27 19:00:00,2144-01-27 19:01:00,2144-01-27 19:12:00,224385,1.0,None,...,4299790,Intubation/Extubation,Task,76.0,0,0,FinishedRunning,1.0,0.0,Intubation
4,10004401,29988601,32773003,42665,2144-01-30 13:00:00,2144-01-30 13:01:00,2144-01-30 13:05:00,224385,1.0,None,...,6177150,Intubation/Extubation,Task,76.0,0,0,FinishedRunning,1.0,0.0,Intubation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8483,19997367,20617667,35616526,10976,2126-04-26 20:14:00,2126-04-26 20:15:00,2126-04-26 21:14:00,224385,1.0,None,...,5095062,Intubation/Extubation,Task,59.0,0,0,FinishedRunning,1.0,0.0,Intubation
8484,19997367,20617667,35616526,27479,2126-05-05 18:20:00,2126-05-05 18:21:00,2126-05-05 18:20:00,224385,1.0,None,...,8462542,Intubation/Extubation,Task,59.0,0,0,FinishedRunning,1.0,0.0,Intubation
8485,19999068,21606769,30143796,29141,2161-08-25 15:34:00,2161-08-25 15:35:00,2161-08-25 15:45:00,224385,1.0,None,...,6823223,Intubation/Extubation,Task,55.8,0,0,FinishedRunning,1.0,0.0,Intubation
8486,19999442,26785317,32336619,59028,2148-11-19 19:00:00,2148-11-19 19:01:00,2148-11-19 21:31:00,224385,1.0,None,...,3602635,Intubation/Extubation,Task,107.5,0,0,FinishedRunning,1.0,0.0,Intubation


noninvasive ventilation

In [ ]:
query = f"""
SELECT label, itemid, linksto
FROM `physionet-data.mimiciv_icu.d_items`
WHERE LOWER(label) LIKE "%non-invasive ventilation%"
"""

df = run_query(query)
df

,label,itemid,linksto
0,Non-invasive Ventilation,225794,procedureevents


We choose "Non-Invasive Ventilation"

In [ ]:
query = f"""
SELECT char.*, dlab.label
FROM `physionet-data.mimiciv_icu.procedureevents` AS char
INNER JOIN `physionet-data.mimiciv_icu.d_items` AS dlab
 ON char.itemid = dlab.itemid
WHERE char.itemid = 225794
ORDER BY subject_id
"""

df_non_invasive_ventilation = run_query(query)
df_non_invasive_ventilation

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,value,valueuom,...,linkorderid,ordercategoryname,ordercategorydescription,patientweight,isopenbag,continueinnextdept,statusdescription,ORIGINALAMOUNT,ORIGINALRATE,label
0,10000980,26913865,39765666,<NA>,2189-06-27 09:01:00,2189-06-27 20:38:00,2189-06-27 20:38:29.047,225794,697.0,min,...,5989583,Ventilation,ContinuousProcess,76.2,1,0,FinishedRunning,697.0,1.0,Non-invasive Ventilation
1,10001884,26184834,37510196,31763,2131-01-12 21:30:00,2131-01-13 04:00:00,2131-01-15 04:07:00.000,225794,390.0,min,...,4809276,Ventilation,ContinuousProcess,65.0,1,0,FinishedRunning,390.0,1.0,Non-invasive Ventilation
2,10002155,20345487,32358465,47007,2131-03-10 00:15:00,2131-03-10 07:59:00,2131-03-10 16:26:00.000,225794,464.0,min,...,1744937,Ventilation,ContinuousProcess,21.1,1,0,FinishedRunning,464.0,1.0,Non-invasive Ventilation
3,10002428,20321825,34807493,99293,2156-04-30 22:54:00,2156-05-02 05:03:00,2156-05-02 09:42:00.000,225794,1809.0,min,...,8673415,Ventilation,ContinuousProcess,55.0,1,0,FinishedRunning,1809.0,1.0,Non-invasive Ventilation
4,10002495,24982426,36753294,6579,2141-05-23 20:15:00,2141-05-24 01:36:00,2141-05-24 06:47:00.000,225794,321.0,min,...,183864,Ventilation,ContinuousProcess,64.1,1,0,FinishedRunning,321.0,1.0,Non-invasive Ventilation
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3006,19997293,20208898,38350444,58760,2123-10-20 02:00:00,2123-10-20 04:03:00,2123-10-20 04:39:00.000,225794,123.0,min,...,6650730,Ventilation,ContinuousProcess,134.5,1,0,FinishedRunning,123.0,1.0,Non-invasive Ventilation
3007,19998330,24096838,33428243,80339,2178-11-28 05:00:00,2178-11-28 09:00:00,2178-11-29 06:14:00.000,225794,240.0,min,...,875619,Ventilation,ContinuousProcess,83.0,1,0,FinishedRunning,240.0,1.0,Non-invasive Ventilation
3008,19998330,24096838,33428243,<NA>,2178-11-29 06:13:00,2178-11-29 21:29:00,2178-11-29 21:29:41.257,225794,916.0,min,...,228938,Ventilation,ContinuousProcess,83.0,1,0,FinishedRunning,916.0,1.0,Non-invasive Ventilation
3009,19999287,20175828,35165301,97774,2197-08-05 05:22:00,2197-08-06 17:32:00,2197-08-06 19:29:00.000,225794,2170.0,min,...,1972780,Ventilation,ContinuousProcess,86.2,1,0,FinishedRunning,2170.0,1.0,Non-invasive Ventilation


In [ ]:
# Convert timestamp columns to datetime objects
df_weaning['endtime'] = pd.to_datetime(df_weaning['endtime'])
df_intubation['starttime'] = pd.to_datetime(df_intubation['starttime'])
df_invasive_ventilation['starttime'] = pd.to_datetime(df_invasive_ventilation['starttime'])
df_non_invasive_ventilation['starttime'] = pd.to_datetime(df_non_invasive_ventilation['starttime'])

# Find patients who underwent weaning
weaning_patients = df_weaning.dropna(subset=['endtime'])

# Initialize a list to store the final results
final_patients = []

# Iterate through weaning patients
for index, row in weaning_patients.iterrows():
    subject_id = row['subject_id']
    hadm_id = row['hadm_id']
    weaning_end_time = row['endtime']

    # Check if there was any intubation within 48 hours of weaning
    intubation_within_48 = df_intubation[(df_intubation['subject_id'] == subject_id) &
                        (df_intubation['hadm_id'] == hadm_id) &
                        (df_intubation['starttime'] <= weaning_end_time) &
                        (weaning_end_time - df_intubation['starttime'] <= pd.Timedelta('48 hours'))]

    # Check if there was any invasive ventilation within 48 hours of weaning
    ventilation_within_48 = df_invasive_ventilation[(df_invasive_ventilation['subject_id'] == subject_id) &
                              (df_invasive_ventilation['hadm_id'] == hadm_id) &
                              (df_invasive_ventilation['starttime'] <= weaning_end_time) &
                              (weaning_end_time - df_invasive_ventilation['starttime'] <= pd.Timedelta('48 hours'))]

    # Check if there was non-invasive ventilation within 48 hours of weaning
    non_invasive_ventilation = df_non_invasive_ventilation[(df_non_invasive_ventilation['subject_id'] == subject_id) &
                                  (df_non_invasive_ventilation['hadm_id'] == hadm_id) &
                                  (df_non_invasive_ventilation['starttime'] <= weaning_end_time) &
                                  (weaning_end_time - df_non_invasive_ventilation['starttime'] <= pd.Timedelta('48 hours'))]

    # Check if there was no intubation or ventilation within 48 hours of weaning, and non-invasive ventilation within 48 hours, then add the patient to the results
    if intubation_within_48.empty and ventilation_within_48.empty and (not non_invasive_ventilation.empty):
        final_patients.append(subject_id)

# Convert the list of patient IDs to a DataFrame
result_df = pd.DataFrame(final_patients, columns=['subject_id'])

# Print the result
print(result_df)

    subject_id
0     10150335
1     10321613
2     10438089
3     10887901
4     11321257
5     11576222
6     11648038
7     12365543
8     12365543
9     12765666
10    12953561
11    13333021
12    13788978
13    13880770
14    14188116
15    14479847
16    15790541
17    15897606
18    15983568
19    16099220
20    16455067
21    16544239
22    16702834
23    17275708
24    17302222
25    17475174
26    18119340
27    18119596
28    18266495
29    18308539
30    18489959
31    18571536
32    18784341
33    19362916
34    19386402
35    19589961
36    19851620


In [ ]:
# convert result_df to a list
patients_list = result_df['subject_id'].to_list()

#### no death within 48 h after weaning

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.mimiciv_hosp.patients`
WHERE dod IS NOT NULL AND subject_id in {tuple(patients_list)}
"""

df_alive = run_query(query)
df_alive_patient_id = df_alive['subject_id'].to_list()
pd.DataFrame(df_alive_patient_id, columns=['subject_id'])

,subject_id
0,10352416
1,16767959
2,15894414
3,12342976
4,10701372
...,...
2938,12033229
2939,16643806
2940,11344751
2941,19149321


### extract the patients' features after getting all the filtered paitents' id

read the filtered patients' id

In [62]:
patients_cohort = pd.read_csv('filtered_patients_dataset.csv', usecols=['subject_id'])
patients_cohort

,subject_id
0,16467047
1,17087880
2,19556965
3,10222191
4,10252795
...,...
5516,11102902
5517,14477446
5518,11230804
5519,15276613


obtain the weaning start time point for all patients(assume invasive ventalition endtime as weaning start time)

In [63]:
query = f"""
SELECT char.subject_id, MIN(char.endtime) AS weaning_start_time, AVG(char.value) AS avg_ventilation_duration_in_minutes
FROM `physionet-data.mimiciv_icu.procedureevents` AS char
WHERE char.itemid = 225792
GROUP BY char.subject_id
ORDER BY char.subject_id
"""

df_first_invasive_ventilation = run_query(query)

# 将时长从分钟转换为天
df_first_invasive_ventilation['avg_ventilation_duration_in_days'] = df_first_invasive_ventilation['avg_ventilation_duration_in_minutes'] / 1440.0
df_first_invasive_ventilation = df_first_invasive_ventilation.drop(columns=['avg_ventilation_duration_in_minutes'])

# 合并到patients_cohort中
patients_cohort = patients_cohort.merge(df_first_invasive_ventilation, on="subject_id", how="left")
patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days
0,16467047,2189-10-20 11:15:00,7.336111
1,17087880,2182-03-08 10:24:00,0.975000
2,19556965,2168-04-30 18:30:00,0.668750
3,10222191,2185-07-10 11:15:00,22.254861
4,10252795,2149-11-16 15:30:00,3.194444
...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167
5517,14477446,2114-02-25 13:11:00,6.563889
5518,11230804,2150-10-24 12:00:00,0.277083
5519,15276613,2160-06-08 16:35:00,0.156250


#### arterial blood gas

Arterial O2 pressure  220224 lowest

Arterial CO2 Pressure  220235 highest

Arterial Base Excess  224828 lowest

Inspired O2 Fraction  223835 higest

In [64]:
# 先获取patients_cohort中的所有subject_id
subject_ids = patients_cohort['subject_id'].tolist()

# 使用这些subject_id来筛选chartevents中的数据
query_o2 = f"""
SELECT subject_id, charttime, valuenum AS arterial_o2_pressure
FROM `physionet-data.mimiciv_icu.chartevents`
WHERE itemid = 220224 AND subject_id IN ({','.join(map(str, subject_ids))})
"""

df_o2 = run_query(query_o2)

# 使用weaning开始时间筛选24小时内的数据
df_o2 = df_o2.merge(patients_cohort[['subject_id', 'weaning_start_time']], on="subject_id", how="inner")
df_o2_filtered = df_o2[(df_o2['charttime'] > df_o2['weaning_start_time'] - pd.Timedelta(hours=24)) &
                       (df_o2['charttime'] <= df_o2['weaning_start_time'])]

# 筛选出每个病人在weaning开始前24小时内的最低arterial_o2_pressure值
df_o2_worst = df_o2_filtered.groupby('subject_id')['arterial_o2_pressure'].min().reset_index()

# 计算所有病人的arterial_o2_pressure的平均值
mean_o2 = df_o2_worst['arterial_o2_pressure'].mean()

# 对于没有arterial_o2_pressure数据的病人，使用平均值填充
missing_data = [{'subject_id': subject, 'arterial_o2_pressure': mean_o2} for subject in subject_ids if subject not in df_o2_worst['subject_id'].unique()]
df_o2_worst = pd.concat([df_o2_worst, pd.DataFrame(missing_data)], ignore_index=True)

# 合并df_o2_worst到patients_cohort
patients_cohort = patients_cohort.merge(df_o2_worst, on='subject_id', how='left')
patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure
0,16467047,2189-10-20 11:15:00,7.336111,100.000000
1,17087880,2182-03-08 10:24:00,0.975000,20.000000
2,19556965,2168-04-30 18:30:00,0.668750,114.000000
3,10222191,2185-07-10 11:15:00,22.254861,36.000000
4,10252795,2149-11-16 15:30:00,3.194444,109.946961
...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000


In [74]:
def add_feature_to_cohort(patients_cohort, itemid, column_name, method='min'):
    subject_ids = patients_cohort['subject_id'].tolist()

    query = f"""
    SELECT subject_id, charttime, valuenum AS {column_name}
    FROM `physionet-data.mimiciv_icu.chartevents`
    WHERE itemid = {itemid} AND subject_id IN ({','.join(map(str, subject_ids))})
    """

    df_feature = run_query(query)

    # 使用weaning开始时间筛选24小时内的数据
    df_feature = df_feature.merge(patients_cohort[['subject_id', 'weaning_start_time']], on="subject_id", how="inner")
    df_feature_filtered = df_feature[(df_feature['charttime'] > df_feature['weaning_start_time'] - pd.Timedelta(hours=24)) &
                                     (df_feature['charttime'] <= df_feature['weaning_start_time'])]

    # 如果column_name不是'arterial_base_excess'，则删除值为0的记录
    if column_name != 'arterial_base_excess':
        df_feature_filtered = df_feature_filtered[df_feature_filtered[column_name] != 0]

    # 根据method选择获取每个病人在weaning开始前24小时内的最高或最低值
    if method == 'min':
        df_feature_worst = df_feature_filtered.groupby('subject_id')[column_name].min().reset_index()
    elif method == 'max':
        df_feature_worst = df_feature_filtered.groupby('subject_id')[column_name].max().reset_index()
    else:
        raise ValueError("Method should be either 'min' or 'max'.")

    # 计算所有病人的平均值
    mean_feature = df_feature_worst[column_name].mean()

    # 对于没有数据的病人，使用平均值填充
    missing_data = [{'subject_id': subject, column_name: mean_feature} for subject in subject_ids if subject not in df_feature_worst['subject_id'].unique()]
    df_feature_worst = pd.concat([df_feature_worst, pd.DataFrame(missing_data)], ignore_index=True)

    # 合并到patients_cohort
    return patients_cohort.merge(df_feature_worst, on='subject_id', how='left')

In [66]:
# 获取每个指标的最差值
patients_cohort = add_feature_to_cohort(patients_cohort, 220235, 'arterial_co2_pressure', method='max')  # 对CO2取最高值
patients_cohort = add_feature_to_cohort(patients_cohort, 224828, 'arterial_base_excess')  # 默认是获取最低值
patients_cohort = add_feature_to_cohort(patients_cohort, 223835, 'inspired_o2_fraction', method='max')  # 获取最高值

patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure,arterial_co2_pressure,arterial_base_excess,inspired_o2_fraction
0,16467047,2189-10-20 11:15:00,7.336111,100.000000,51.000000,-1.000000,50.0
1,17087880,2182-03-08 10:24:00,0.975000,20.000000,75.000000,-18.000000,100.0
2,19556965,2168-04-30 18:30:00,0.668750,114.000000,37.000000,-20.000000,40.0
3,10222191,2185-07-10 11:15:00,22.254861,36.000000,49.000000,0.000000,40.0
4,10252795,2149-11-16 15:30:00,3.194444,109.946961,475.305562,-2.995491,40.0
...,...,...,...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000,48.000000,0.000000,50.0
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961,475.305562,-2.995491,35.0
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000,36.000000,0.000000,40.0
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000,57.000000,-4.000000,100.0


#### full blood count

WBC 220546

platelet 227457

Hemoglobin	220228

In [67]:
# 使用定义的函数为patients_cohort添加新的特征
patients_cohort = add_feature_to_cohort(patients_cohort, 220546, "WBC", method='max')  # 对WBC取最高值
patients_cohort = add_feature_to_cohort(patients_cohort, 227457, "platelet")
patients_cohort = add_feature_to_cohort(patients_cohort, 220228, "Hemoglobin")
patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure,arterial_co2_pressure,arterial_base_excess,inspired_o2_fraction,WBC,platelet,Hemoglobin
0,16467047,2189-10-20 11:15:00,7.336111,100.000000,51.000000,-1.000000,50.0,15.0,209.0,8.7
1,17087880,2182-03-08 10:24:00,0.975000,20.000000,75.000000,-18.000000,100.0,7.5,68.0,5.7
2,19556965,2168-04-30 18:30:00,0.668750,114.000000,37.000000,-20.000000,40.0,28.9,226.0,7.8
3,10222191,2185-07-10 11:15:00,22.254861,36.000000,49.000000,0.000000,40.0,24.1,423.0,7.8
4,10252795,2149-11-16 15:30:00,3.194444,109.946961,475.305562,-2.995491,40.0,7.8,110.0,10.1
...,...,...,...,...,...,...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000,48.000000,0.000000,50.0,16.9,281.0,10.8
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961,475.305562,-2.995491,35.0,9.9,316.0,9.0
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000,36.000000,0.000000,40.0,16.3,176.0,12.4
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000,57.000000,-4.000000,100.0,13.9,181.0,11.4


#### vital signs

Heart rate 220045 highest

Respiratory Rate	220210 highest

Arterial Blood Pressure mean	220052 lowest

Temperature Celsius	223762 highest

In [68]:
# 获取每个指标的最差值，并合并到patients_cohort
patients_cohort = add_feature_to_cohort(patients_cohort, 220045, 'Heart_rate', method='max')  # 获取最高值

patients_cohort = add_feature_to_cohort(patients_cohort, 220210, 'Respiratory_Rate', method='max')  # 获取最高值

patients_cohort = add_feature_to_cohort(patients_cohort, 220052, 'Arterial_BP_mean', method='min')  # 获取最低值

patients_cohort = add_feature_to_cohort(patients_cohort, 223762, 'Temperature_Celsius', method='max')  # 获取最高值

patients_cohort


,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure,arterial_co2_pressure,arterial_base_excess,inspired_o2_fraction,WBC,platelet,Hemoglobin,Heart_rate,Respiratory_Rate,Arterial_BP_mean,Temperature_Celsius
0,16467047,2189-10-20 11:15:00,7.336111,100.000000,51.000000,-1.000000,50.0,15.0,209.0,8.7,111.0,34.0,57.322394,37.541654
1,17087880,2182-03-08 10:24:00,0.975000,20.000000,75.000000,-18.000000,100.0,7.5,68.0,5.7,132.0,32.0,59.000000,37.541654
2,19556965,2168-04-30 18:30:00,0.668750,114.000000,37.000000,-20.000000,40.0,28.9,226.0,7.8,126.0,21.0,45.000000,37.541654
3,10222191,2185-07-10 11:15:00,22.254861,36.000000,49.000000,0.000000,40.0,24.1,423.0,7.8,87.0,37.0,57.322394,37.541654
4,10252795,2149-11-16 15:30:00,3.194444,109.946961,475.305562,-2.995491,40.0,7.8,110.0,10.1,77.0,21.0,60.000000,37.541654
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000,48.000000,0.000000,50.0,16.9,281.0,10.8,111.0,19.0,54.000000,37.541654
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961,475.305562,-2.995491,35.0,9.9,316.0,9.0,84.0,27.0,61.000000,37.541654
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000,36.000000,0.000000,40.0,16.3,176.0,12.4,74.0,25.0,78.000000,37.541654
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000,57.000000,-4.000000,100.0,13.9,181.0,11.4,100.0,21.0,64.000000,38.700000


#### laboratory index

Creatinine (serum)	220615 highest

In [69]:
patients_cohort = add_feature_to_cohort(patients_cohort, 220615, 'Creatinine', method='max')  # 获取Creatinine的最高值

patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure,arterial_co2_pressure,arterial_base_excess,inspired_o2_fraction,WBC,platelet,Hemoglobin,Heart_rate,Respiratory_Rate,Arterial_BP_mean,Temperature_Celsius,Creatinine
0,16467047,2189-10-20 11:15:00,7.336111,100.000000,51.000000,-1.000000,50.0,15.0,209.0,8.7,111.0,34.0,57.322394,37.541654,1.0
1,17087880,2182-03-08 10:24:00,0.975000,20.000000,75.000000,-18.000000,100.0,7.5,68.0,5.7,132.0,32.0,59.000000,37.541654,2.3
2,19556965,2168-04-30 18:30:00,0.668750,114.000000,37.000000,-20.000000,40.0,28.9,226.0,7.8,126.0,21.0,45.000000,37.541654,2.5
3,10222191,2185-07-10 11:15:00,22.254861,36.000000,49.000000,0.000000,40.0,24.1,423.0,7.8,87.0,37.0,57.322394,37.541654,1.8
4,10252795,2149-11-16 15:30:00,3.194444,109.946961,475.305562,-2.995491,40.0,7.8,110.0,10.1,77.0,21.0,60.000000,37.541654,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000,48.000000,0.000000,50.0,16.9,281.0,10.8,111.0,19.0,54.000000,37.541654,0.5
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961,475.305562,-2.995491,35.0,9.9,316.0,9.0,84.0,27.0,61.000000,37.541654,0.6
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000,36.000000,0.000000,40.0,16.3,176.0,12.4,74.0,25.0,78.000000,37.541654,0.8
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000,57.000000,-4.000000,100.0,13.9,181.0,11.4,100.0,21.0,64.000000,38.700000,0.7


#### some other left features

age

gender

In [70]:
query_age_gender = """
SELECT subject_id, anchor_age AS age, gender
FROM `physionet-data.mimiciv_hosp.patients`
WHERE anchor_age >= 18
"""

age_gender_df = run_query(query_age_gender)

# 合并年龄和性别到patients_cohort
patients_cohort = patients_cohort.merge(age_gender_df, on='subject_id', how='left')
patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure,arterial_co2_pressure,arterial_base_excess,inspired_o2_fraction,WBC,platelet,Hemoglobin,Heart_rate,Respiratory_Rate,Arterial_BP_mean,Temperature_Celsius,Creatinine,age,gender
0,16467047,2189-10-20 11:15:00,7.336111,100.000000,51.000000,-1.000000,50.0,15.0,209.0,8.7,111.0,34.0,57.322394,37.541654,1.0,74,F
1,17087880,2182-03-08 10:24:00,0.975000,20.000000,75.000000,-18.000000,100.0,7.5,68.0,5.7,132.0,32.0,59.000000,37.541654,2.3,59,F
2,19556965,2168-04-30 18:30:00,0.668750,114.000000,37.000000,-20.000000,40.0,28.9,226.0,7.8,126.0,21.0,45.000000,37.541654,2.5,69,F
3,10222191,2185-07-10 11:15:00,22.254861,36.000000,49.000000,0.000000,40.0,24.1,423.0,7.8,87.0,37.0,57.322394,37.541654,1.8,77,M
4,10252795,2149-11-16 15:30:00,3.194444,109.946961,475.305562,-2.995491,40.0,7.8,110.0,10.1,77.0,21.0,60.000000,37.541654,1.1,72,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000,48.000000,0.000000,50.0,16.9,281.0,10.8,111.0,19.0,54.000000,37.541654,0.5,69,F
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961,475.305562,-2.995491,35.0,9.9,316.0,9.0,84.0,27.0,61.000000,37.541654,0.6,88,F
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000,36.000000,0.000000,40.0,16.3,176.0,12.4,74.0,25.0,78.000000,37.541654,0.8,65,M
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000,57.000000,-4.000000,100.0,13.9,181.0,11.4,100.0,21.0,64.000000,38.700000,0.7,54,M


In [71]:
dod_gender = """
SELECT subject_id, dod
FROM `physionet-data.mimiciv_hosp.patients`
"""

dod_df = run_query(dod_gender)

# 合并年龄和性别到patients_cohort
patients_cohort = patients_cohort.merge(dod_df, on='subject_id', how='left')
patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure,arterial_co2_pressure,arterial_base_excess,inspired_o2_fraction,WBC,platelet,Hemoglobin,Heart_rate,Respiratory_Rate,Arterial_BP_mean,Temperature_Celsius,Creatinine,age,gender,dod
0,16467047,2189-10-20 11:15:00,7.336111,100.000000,51.000000,-1.000000,50.0,15.0,209.0,8.7,111.0,34.0,57.322394,37.541654,1.0,74,F,2189-11-21
1,17087880,2182-03-08 10:24:00,0.975000,20.000000,75.000000,-18.000000,100.0,7.5,68.0,5.7,132.0,32.0,59.000000,37.541654,2.3,59,F,2182-03-08
2,19556965,2168-04-30 18:30:00,0.668750,114.000000,37.000000,-20.000000,40.0,28.9,226.0,7.8,126.0,21.0,45.000000,37.541654,2.5,69,F,2168-04-30
3,10222191,2185-07-10 11:15:00,22.254861,36.000000,49.000000,0.000000,40.0,24.1,423.0,7.8,87.0,37.0,57.322394,37.541654,1.8,77,M,2185-09-01
4,10252795,2149-11-16 15:30:00,3.194444,109.946961,475.305562,-2.995491,40.0,7.8,110.0,10.1,77.0,21.0,60.000000,37.541654,1.1,72,M,2149-11-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000,48.000000,0.000000,50.0,16.9,281.0,10.8,111.0,19.0,54.000000,37.541654,0.5,69,F,NaT
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961,475.305562,-2.995491,35.0,9.9,316.0,9.0,84.0,27.0,61.000000,37.541654,0.6,88,F,2114-02-27
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000,36.000000,0.000000,40.0,16.3,176.0,12.4,74.0,25.0,78.000000,37.541654,0.8,65,M,NaT
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000,57.000000,-4.000000,100.0,13.9,181.0,11.4,100.0,21.0,64.000000,38.700000,0.7,54,M,NaT


#### classify WF and WS

In [79]:
def classify_weaning_success(patients_cohort):
    subject_ids = patients_cohort['subject_id'].tolist()

    # 获取非侵入性通气的记录
    non_invasive_vent_query = f"""
    SELECT subject_id, starttime, value
    FROM `physionet-data.mimiciv_icu.procedureevents`
    WHERE itemid = 225794 AND subject_id IN ({','.join(map(str, subject_ids))})
    """
    non_invasive_vent_df = run_query(non_invasive_vent_query)

    # 获取侵入性通气的记录
    invasive_vent_query = f"""
    SELECT subject_id, starttime
    FROM `physionet-data.mimiciv_icu.procedureevents`
    WHERE itemid = 225792 AND subject_id IN ({','.join(map(str, subject_ids))})
    """
    invasive_vent_df = run_query(invasive_vent_query)

    # 根据条件分类
    def classify_row(row):
        weaning_time = row['weaning_start_time']
        subject = row['subject_id']

        # 检查条件 (c)
        non_inv_vent_records = non_invasive_vent_df[non_invasive_vent_df['subject_id'] == subject]
        within_48_hours_records = non_inv_vent_records[(non_inv_vent_records['starttime'] > weaning_time) &
                                                       (non_inv_vent_records['starttime'] <= weaning_time + pd.Timedelta(hours=48))]
        total_duration = within_48_hours_records['value'].sum()
        if total_duration >= 48*60:  # 将时间从分钟转换为48小时
            return "WF"

        # 检查条件 (a)
        inv_vent_records = invasive_vent_df[invasive_vent_df['subject_id'] == subject]
        if any((inv_vent_records['starttime'] > weaning_time) &
               (inv_vent_records['starttime'] <= weaning_time + pd.Timedelta(hours=48))):
            return "WF"

        # 检查条件 (b)
        death_date = row['dod']
        if not pd.isnull(death_date) and pd.Timestamp(death_date) <= weaning_time + pd.Timedelta(hours=48):
            return "WF"

        return "WS"

    patients_cohort['weaning_class'] = patients_cohort.apply(classify_row, axis=1)
    return patients_cohort

patients_cohort = classify_weaning_success(patients_cohort)
patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure,arterial_co2_pressure,arterial_base_excess,inspired_o2_fraction,WBC,platelet,Hemoglobin,...,Temperature_Celsius,Creatinine,age,gender,dod,PEEP_Level,SpO2,Anion_gap,Urine_output,weaning_class
0,16467047,2189-10-20 11:15:00,7.336111,100.000000,51.000000,-1.000000,50.0,15.0,209.0,8.7,...,37.541654,1.0,74,F,2189-11-21,7.456549,85.0,15.0,575.51594,WS
1,17087880,2182-03-08 10:24:00,0.975000,20.000000,75.000000,-18.000000,100.0,7.5,68.0,5.7,...,37.541654,2.3,59,F,2182-03-08,6.000000,85.0,32.0,575.51594,WF
2,19556965,2168-04-30 18:30:00,0.668750,114.000000,37.000000,-20.000000,40.0,28.9,226.0,7.8,...,37.541654,2.5,69,F,2168-04-30,7.000000,85.0,48.0,575.51594,WF
3,10222191,2185-07-10 11:15:00,22.254861,36.000000,49.000000,0.000000,40.0,24.1,423.0,7.8,...,37.541654,1.8,77,M,2185-09-01,7.456549,85.0,16.0,575.51594,WF
4,10252795,2149-11-16 15:30:00,3.194444,109.946961,475.305562,-2.995491,40.0,7.8,110.0,10.1,...,37.541654,1.1,72,M,2149-11-19,5.000000,85.0,12.0,575.51594,WS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000,48.000000,0.000000,50.0,16.9,281.0,10.8,...,37.541654,0.5,69,F,NaT,7.456549,85.0,12.0,1600.00000,WS
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961,475.305562,-2.995491,35.0,9.9,316.0,9.0,...,37.541654,0.6,88,F,2114-02-27,7.456549,88.0,10.0,575.51594,WF
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000,36.000000,0.000000,40.0,16.3,176.0,12.4,...,37.541654,0.8,65,M,NaT,5.000000,88.0,14.0,150.00000,WS
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000,57.000000,-4.000000,100.0,13.9,181.0,11.4,...,38.700000,0.7,54,M,NaT,7.456549,85.0,13.0,650.00000,WS


#### additional
226566     Urine and GU Irrigant Out minimum

224700       Total PEEP Level highest

226253       SpO2 Desat Limit minimum

Anion gap	227073 highest

In [76]:
def add_feature_to_cohort(patients_cohort, itemid, column_name, table_name='chartevents', method='min'):
    subject_ids = patients_cohort['subject_id'].tolist()

    query = f"""
    SELECT subject_id, charttime, value AS {column_name}
    FROM `physionet-data.mimiciv_icu.{table_name}`
    WHERE itemid = {itemid} AND subject_id IN ({','.join(map(str, subject_ids))})
    """

    df_feature = run_query(query)

    # 使用weaning开始时间筛选24小时内的数据
    df_feature = df_feature.merge(patients_cohort[['subject_id', 'weaning_start_time']], on="subject_id", how="inner")
    df_feature_filtered = df_feature[(df_feature['charttime'] > df_feature['weaning_start_time'] - pd.Timedelta(hours=24)) &
                                     (df_feature['charttime'] <= df_feature['weaning_start_time'])]

    # 如果column_name不是'arterial_base_excess'，则删除值为0的记录
    if column_name != 'arterial_base_excess':
        df_feature_filtered = df_feature_filtered[df_feature_filtered[column_name] != 0]

    # 根据method选择获取每个病人在weaning开始前24小时内的最高或最低值
    if method == 'min':
        df_feature_worst = df_feature_filtered.groupby('subject_id')[column_name].min().reset_index()
    elif method == 'max':
        df_feature_worst = df_feature_filtered.groupby('subject_id')[column_name].max().reset_index()
    else:
        raise ValueError("Method should be either 'min' or 'max'.")

    # 计算所有病人的平均值
    mean_feature = df_feature_worst[column_name].mean()

    # 对于没有数据的病人，使用平均值填充
    missing_data = [{'subject_id': subject, column_name: mean_feature} for subject in subject_ids if subject not in df_feature_worst['subject_id'].unique()]
    df_feature_worst = pd.concat([df_feature_worst, pd.DataFrame(missing_data)], ignore_index=True)

    # 合并到patients_cohort
    return patients_cohort.merge(df_feature_worst, on='subject_id', how='left')


In [77]:
patients_cohort = add_feature_to_cohort(patients_cohort, 226627, 'Urine_output',table_name = 'outputevents', method='min')
#patients_cohort = add_feature_to_cohort(patients_cohort, 224700, 'PEEP_Level', method='max')
#patients_cohort = add_feature_to_cohort(patients_cohort, 226253, 'SpO2', method='min')
#patients_cohort = add_feature_to_cohort(patients_cohort, 227073, 'Anion_gap', method='max')
patients_cohort

,subject_id,weaning_start_time,avg_ventilation_duration_in_days,arterial_o2_pressure,arterial_co2_pressure,arterial_base_excess,inspired_o2_fraction,WBC,platelet,Hemoglobin,...,Arterial_BP_mean,Temperature_Celsius,Creatinine,age,gender,dod,PEEP_Level,SpO2,Anion_gap,Urine_output
0,16467047,2189-10-20 11:15:00,7.336111,100.000000,51.000000,-1.000000,50.0,15.0,209.0,8.7,...,57.322394,37.541654,1.0,74,F,2189-11-21,7.456549,85.0,15.0,575.51594
1,17087880,2182-03-08 10:24:00,0.975000,20.000000,75.000000,-18.000000,100.0,7.5,68.0,5.7,...,59.000000,37.541654,2.3,59,F,2182-03-08,6.000000,85.0,32.0,575.51594
2,19556965,2168-04-30 18:30:00,0.668750,114.000000,37.000000,-20.000000,40.0,28.9,226.0,7.8,...,45.000000,37.541654,2.5,69,F,2168-04-30,7.000000,85.0,48.0,575.51594
3,10222191,2185-07-10 11:15:00,22.254861,36.000000,49.000000,0.000000,40.0,24.1,423.0,7.8,...,57.322394,37.541654,1.8,77,M,2185-09-01,7.456549,85.0,16.0,575.51594
4,10252795,2149-11-16 15:30:00,3.194444,109.946961,475.305562,-2.995491,40.0,7.8,110.0,10.1,...,60.000000,37.541654,1.1,72,M,2149-11-19,5.000000,85.0,12.0,575.51594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5516,11102902,2143-02-16 09:20:00,0.304167,94.000000,48.000000,0.000000,50.0,16.9,281.0,10.8,...,54.000000,37.541654,0.5,69,F,NaT,7.456549,85.0,12.0,1600.00000
5517,14477446,2114-02-25 13:11:00,6.563889,109.946961,475.305562,-2.995491,35.0,9.9,316.0,9.0,...,61.000000,37.541654,0.6,88,F,2114-02-27,7.456549,88.0,10.0,575.51594
5518,11230804,2150-10-24 12:00:00,0.277083,315.000000,36.000000,0.000000,40.0,16.3,176.0,12.4,...,78.000000,37.541654,0.8,65,M,NaT,5.000000,88.0,14.0,150.00000
5519,15276613,2160-06-08 16:35:00,0.156250,99.000000,57.000000,-4.000000,100.0,13.9,181.0,11.4,...,64.000000,38.700000,0.7,54,M,NaT,7.456549,85.0,13.0,650.00000


In [80]:
patients_cohort.to_csv('filtered_patients_dataset_v2.csv', index=False)